In [42]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime
import os

In [43]:
mayor= "2020-03"
menor = "2019-02"

In [44]:
data = querydbtopandas(f"""select id_prestador_efector, 
    a14.id_afiliado,
    desc_pre_nombre, 
    CONVERT(DATETIME,d.exp_mes, "mm-YYYY") exp_mes,
    sum(i_ft_cantidad) cantidad,
    sum(i_ft_importe) importe,
    id_medicamento,
    Q.deno
from dba.ft_costo_medico A
    left JOIN dba.d_pre_prestador B
        ON a.id_prestador_efector = b.id_pre_prestador
    LEFT JOIN dba.fecha_mes D
        ON a.id_tie_fecha_liquidacion = d.id_Tie_mes
    left join DBA.afi_afiliados a14
         on a.id_afiliado = a14.id_afiliado
    left JOIN dba.prestad_costo_centros E
        ON A.id_prestador_efector = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
    LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
where CONVERT(DATETIME,d.exp_mes, "mm-YYYY") >= date("{menor}") and CONVERT(DATETIME,d.exp_mes, "mm-YYYY") < date("{mayor}")
and id_prestador_efector not in (0)
and tbl_origen = 'receta_medicamento'
group by id_prestador_efector, desc_pre_nombre, a14.id_afiliado,Q.deno,
d.exp_mes,
    id_medicamento,
i_ft_cantidad, i_ft_importe""")

In [45]:
data.exp_mes = data.exp_mes.astype(str).str[:7]

In [46]:
for i in data.exp_mes.unique():
    df = data[data["exp_mes"] == i]
    
    #filtro por tope minimo de 23 recetas por prestador por mes
    #cantidad_ordenes = df.groupby(["id_prestador_efector"])["cantidad"].sum()
    #df = df[df["id_prestador_efector"].isin((cantidad_ordenes[cantidad_ordenes >= 23]).index)]
    
    #calculo metrica
    importe = df.groupby(["id_prestador_efector","desc_pre_nombre", "deno" ])["importe"].sum()

    cantidad_ordenes = df.groupby(["id_prestador_efector","desc_pre_nombre", "deno" ])["cantidad"].sum()

    cantidad_afiliados = df.groupby(["id_prestador_efector","desc_pre_nombre", "deno" ])["id_afiliado"].nunique()

    df = pd.concat([importe,cantidad_ordenes,cantidad_afiliados],axis=1).reset_index()

    df["promedio por afiliado"] = df["importe"] / df["id_afiliado"]
    df["promedio por receta"] = df["importe"] / df["cantidad"]

    suma_especialidad = df.groupby("deno")["promedio por afiliado"].sum()
    desv_stan_especialidad = df.groupby("deno")['promedio por afiliado'].std().rename("desvios por especialidad").reset_index()
    
    num_prestador_esp = df.groupby("deno")["desc_pre_nombre"].nunique()

    media_especialidad = (suma_especialidad/num_prestador_esp).reset_index().rename(columns= {0: "media por especialidad"})
    
    
    df = df.merge(media_especialidad).merge(desv_stan_especialidad)

    df["Diferencia respecto de la media"] = ((df["promedio por afiliado"] / df["media por especialidad"]) -1 )*100

    df["desvios de la media de su sp"] = (df["promedio por afiliado"] - df["media por especialidad"]) /  df["desvios por especialidad"]

    df = df[['id_prestador_efector', 'desc_pre_nombre', 'deno',  'media por especialidad',  'Diferencia respecto de la media',
            'id_afiliado', 'cantidad', 'promedio por afiliado',
           'promedio por receta', 'importe' , "desvios de la media de su sp"]]
    
    df = df.round(2)

    df.columns = ["N° Prestador", "Médico", "Especialidad", "Media por especialidad", "Diferencia respecto de la media", "Afil. Usuarios", "Recetas", "Promedio por afiliado", "Promedio por receta", "A cargo de SMMP", "desvios de la media de su sp"]
    
    df.to_parquet(f'datos_farmacia/{i}.gzip',compression='gzip') 


In [7]:
fechas = [i for i in os.listdir('datos_farmacia/') if len(i) == 12]
completo = pd.DataFrame()
for i in fechas:
    completo = pd.concat([completo, pd.read_parquet(f'datos_farmacia/{i}')])

In [12]:
completo["N° Prestador"].nunique()

16344

In [13]:
pd.Series(completo["N° Prestador"].unique()).to_csv("prestadores_unicos_farmacia.csv")